In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy as stats
import scipy as s
from scipy import stats
# import seaborn as sns
plt.rcParams['font.size'] = 18
#plt.rcParams['font.family'] = "serif"
tdir = 'in'
major = 5.0
minor = 3.0
plt.rcParams['xtick.direction'] = tdir
plt.rcParams['ytick.direction'] = tdir
plt.rcParams['xtick.major.size'] = major
plt.rcParams['xtick.minor.size'] = minor
plt.rcParams['ytick.major.size'] = major
plt.rcParams['ytick.minor.size'] = minor

In [2]:
def get_average_all_exps(result_dict, get_exps_min=False):
    
    multiexps = {}
    for k, runs in result_dict.items():
        #print(k)
        #for runs in precs_dict[k]:
        #print(runs)
        for ind, run in enumerate(runs):
            #print(run)
            try:
                if ind in multiexps.keys():
                    multiexps[ind].append(run)
                else:
                    multiexps[ind] = [run]
            except: continue


    exps_mean = []
    exps_min = []
    for exp in multiexps.keys():
        exps_mean.append(np.mean(multiexps[exp]))
        exps_min.append(np.min(multiexps[exp]))
        #print(np.mean(multiexps[exp]), np.std(multiexps[exp]))

    assert len(exps_mean) == len(exps_min)

    #print(f'Mean {np.mean(exps_mean)* 100:.2f}\u00B1{np.std(exps_mean)* 100:.2f}')
    #print(f'Min {np.mean(exps_min)* 100:.2f}\u00B1{np.std(exps_min)* 100:.2f}')
    
    if get_exps_min:
        return exps_mean, exps_min
    else:
        return exps_mean 
    
    
    
def ifs_results(scenario, replay_config, ifs_option='ratio', joint=False, get_min=False,
                cnt_rate=0.1, anomaly_perct=0.5, min_samples=1, memory_budget=1000):

    #target_classes = 100
    
    results_dir = './Submission_Task/'

    if replay_config == 'grs':
        if joint:
            result_file = str(replay_config)  + '_joint_results.txt'
        else:
            result_file = str(replay_config)  + '_' +\
                    str(memory_budget)  + '_results.txt'
            
    elif replay_config == 'ifs':
        if min_samples > 1:
            result_file = 'ifs_' + str(ifs_option) + '_' + str(memory_budget) + '_' + str(min_samples) + '_results.txt'
        else:
            result_file = 'ifs_' + str(ifs_option) + '_' + str(memory_budget) + '_results.txt'
    elif replay_config == 'aws':
        if min_samples > 1:
            result_file = 'aws_' + str(ifs_option) + '_' + str(memory_budget) + '_' + str(min_samples) + '_results.txt'
        else:
            result_file = 'aws_' + str(ifs_option) + '_' + str(memory_budget) + '_results.txt'
        
    else:
        result_file = str(replay_config)  + '_' +\
                    str(memory_budget)  + '_results.txt'

        
    result_path = results_dir + result_file 
    #print(result_path)
    
    accs_dict = {}
    precs_dict = {}
    recalls_dict = {}
    f1_dict = {}

    with open(result_path) as f:
            lines = f.readlines()
            #print(lines)
            for line in lines:
                #print(line)
                infos = line.split('\t')
                #print(infos)
                task = infos[0]
                #print(int(task))
                acc, prec, recal, f1 = float(infos[1]), float(infos[2]), float(infos[3]), float(infos[4])

                if task in accs_dict.keys():
                    accs_dict[task].append(acc)
                    precs_dict[task].append(prec)
                    recalls_dict[task].append(recal)
                    f1_dict[task].append(f1)
                else:
                    accs_dict[task] = [acc]
                    precs_dict[task] = [prec]
                    recalls_dict[task] = [recal]
                    f1_dict[task] = [f1]

    #print(accs_dict)
    
    #accs_all_task = []
    accs_all = get_average_all_exps(accs_dict, get_exps_min=False)
    precs_all, precs_all_min = get_average_all_exps(precs_dict, get_exps_min=True)
    
    recalls_all =get_average_all_exps(recalls_dict, get_exps_min=False)
    f1_all = get_average_all_exps(f1_dict, get_exps_min=False)
    
    
    precs_all_task = []
    f1_all_task = []
    for k, v in precs_dict.items():
        precs_all_task.append(v)
        f1_all_task.append(f1_dict[k])
        

    #print()
    if get_min:
        if replay_config == 'grs':
            if joint:
                print(f'\t {str(replay_config)}-Joint\tprec-min\t{np.mean(precs_all_min)* 100:.1f}$\pm${np.std(precs_all_min) * 100:.1f}')
            else:
                print(f'\t {str(replay_config)}-{str(memory_budget)}\tprec-min\t{np.mean(precs_all_min)* 100:.1f}$\pm${np.std(precs_all_min) * 100:.1f}')
                
        if replay_config == 'ifs' or replay_config == 'aws':
            print(f'\t{str(replay_config)}-{str(ifs_option)}-{str(memory_budget)}\t prec-min\t{np.mean(precs_all_min)* 100:.1f}\u00B1{np.std(precs_all_min) * 100:.1f} \n')
    else:
        if replay_config == 'grs':
            if joint:
                #print(f'\t\t {str(replay_config)}-Joint \n#################\t###################\t###################\t###################\nprec   {np.mean(precs_all)* 100:.2f}\u00B1{np.std(precs_all) * 100:.2f} \tprec-min\t{np.mean(precs_all_min)* 100:.2f}\u00B1{np.std(precs_all_min) * 100:.2f} \trecall\t {np.mean(recalls_all)* 100:.2f}\u00B1{np.std(recalls_all) * 100:.2f} \tf1\t{np.mean(f1_all)* 100:.2f}\u00B1{np.std(f1_all) * 100:.2f} \n#################\t###################\t###################\t###################')
                #print(f'\t {str(replay_config)}-Joint\tprec-min\t{np.mean(precs_all_min)* 100:.1f}$\pm${np.std(precs_all_min) * 100:.1f}')
                print(f'\t {str(replay_config)}-Joint\tprec\t{np.mean(precs_all)* 100:.1f}$\pm${np.std(precs_all) * 100:.1f}')
            else:
                print(f'\t {str(replay_config)}-{str(memory_budget)}\tprec\t{np.mean(precs_all)* 100:.1f}$\pm${np.std(precs_all) * 100:.1f}')
                #print(f'\t {str(replay_config)}-{str(memory_budget)}\tprec-min\t{np.mean(precs_all_min)* 100:.1f}$\pm${np.std(precs_all_min) * 100:.1f}')
                #print(f'\t\t {str(replay_config)}-{str(memory_budget)}\n#################\t###################\t###################\t###################\nprec   {np.mean(precs_all)* 100:.2f}\u00B1{np.std(precs_all) * 100:.2f} \tprec-min\t{np.mean(precs_all_min)* 100:.2f}\u00B1{np.std(precs_all_min) * 100:.2f} \trecall\t {np.mean(recalls_all)* 100:.2f}\u00B1{np.std(recalls_all) * 100:.2f} \tf1\t{np.mean(f1_all)* 100:.2f}\u00B1{np.std(f1_all) * 100:.2f} \n#################\t###################\t###################\t###################')

        if replay_config == 'ifs' or replay_config == 'aws':
            print(f'\t{str(replay_config)}-{str(ifs_option)}-{str(memory_budget)}\tprec\t{np.mean(precs_all)* 100:.1f}$\pm${np.std(precs_all) * 100:.1f}')
            #print(f'\t\t {str(replay_config)}-{str(ifs_option)}-{str(memory_budget)}\n#################\t###################\t###################\t###################\nprec   {np.mean(precs_all)* 100:.2f}\u00B1{np.std(precs_all) * 100:.2f} \tprec-min\t{np.mean(precs_all_min)* 100:.2f}\u00B1{np.std(precs_all_min) * 100:.2f} \trecall\t {np.mean(recalls_all)* 100:.2f}\u00B1{np.std(recalls_all) * 100:.2f} \tf1\t{np.mean(f1_all)* 100:.2f}\u00B1{np.std(f1_all) * 100:.2f} \n#################\t###################\t###################\t###################')
            #print(f'\t{str(replay_config)}-{str(ifs_option)}-{str(memory_budget)}\t prec-min\t{np.mean(precs_all_min)* 100:.2f}\u00B1{np.std(precs_all_min) * 100:.2f} \n')
            #print(f'\t{str(replay_config)}-{str(ifs_option)}-{str(memory_budget)}\tprec-min\t{np.mean(precs_all_min)* 100:.1f}$\pm${np.std(precs_all_min) * 100:.1f}')
        #print()
    
    
    return  np.array(precs_all_task), np.array(accs_all), np.array(precs_all), np.array(recalls_all), np.array(f1_all_task), np.array(f1_all)


In [3]:
grs_joint, _, _, _, _, _ = ifs_results('task', replay_config='grs', joint=True) 
grs_500, _, _, _, _, _ = ifs_results('task', replay_config='grs', memory_budget=500)
grs_1K, _, _, _, _, _ = ifs_results('task', replay_config='grs', memory_budget=1000)
grs_5K, _, _, _, _, _ = ifs_results('task', replay_config='grs', memory_budget=5000)
grs_10K, _, _, _, _, _ = ifs_results('task', replay_config='grs', memory_budget=10000)
grs_15K, _, _, _, _, _ = ifs_results('task', replay_config='grs', memory_budget=15000)
grs_20K, _, _, _, _, _ = ifs_results('task', replay_config='grs', memory_budget=20000);


	 grs-Joint	prec	92.7$\pm$0.7
	 grs-500	prec	86.0$\pm$3.6
	 grs-1000	prec	85.3$\pm$0.1
	 grs-5000	prec	89.4$\pm$1.6
	 grs-10000	prec	91.1$\pm$0.7
	 grs-15000	prec	90.8$\pm$0.9
	 grs-20000	prec	91.6$\pm$1.6


/home/mr6564/anaconda/envs/PyTorch/lib/python3.7/site-packages/ipykernel_launcher.py:143: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


In [5]:
ifsR_500, _, _, _, _, _ = ifs_results('task', replay_config='ifs', ifs_option='ratio', memory_budget=100)
ifsR_500, _, _, _, _, _ = ifs_results('task', replay_config='ifs', ifs_option='ratio', memory_budget=500)
ifsR_1K, _, _, _, _, _ = ifs_results('task', replay_config='ifs', ifs_option='ratio', memory_budget=1000)
ifsR_5K, _, _, _, _, _ = ifs_results('task', replay_config='ifs', ifs_option='ratio', memory_budget=5000)
ifsR_10K, _, _, _, _, _ = ifs_results('task', replay_config='ifs', ifs_option='ratio', memory_budget=10000)
ifsR_15K, _, _, _, _, _ = ifs_results('task', replay_config='ifs', ifs_option='ratio', memory_budget=15000)
ifsR_20K, _, _, _, _, _ = ifs_results('task', replay_config='ifs', ifs_option='ratio', memory_budget=20000)

	ifs-ratio-100	prec	88.2$\pm$0.0
	ifs-ratio-500	prec	86.0$\pm$1.0
	ifs-ratio-1000	prec	87.3$\pm$0.5
	ifs-ratio-5000	prec	89.8$\pm$0.6
	ifs-ratio-10000	prec	91.6$\pm$0.8
	ifs-ratio-15000	prec	91.9$\pm$1.1
	ifs-ratio-20000	prec	91.7$\pm$0.6


In [21]:
ifsU_500, _, _, _, _, _ = ifs_results('task', replay_config='ifs', ifs_option='uniform', memory_budget=500)
ifsU_1K, _, _, _, _, _ = ifs_results('task', replay_config='ifs', ifs_option='uniform', memory_budget=1000)
ifsU_5K, _, _, _, _, _ = ifs_results('task', replay_config='ifs', ifs_option='uniform', memory_budget=5000)
ifsU_10K, _, _, _, _, _ = ifs_results('task', replay_config='ifs', ifs_option='uniform', memory_budget=10000)
ifsU_15K, _, _, _, _, _ = ifs_results('task', replay_config='ifs', ifs_option='uniform', memory_budget=15000)
ifsU_20K, _, _, _, _, _ = ifs_results('task', replay_config='ifs', ifs_option='uniform', memory_budget=20000)


	ifs-uniform-500	prec	88.7$\pm$0.7
	ifs-uniform-1000	prec	90.2$\pm$2.0
	ifs-uniform-5000	prec	90.6$\pm$1.0
	ifs-uniform-10000	prec	92.3$\pm$1.1
	ifs-uniform-15000	prec	90.6$\pm$0.8
	ifs-uniform-20000	prec	91.3$\pm$0.5


In [25]:
ifsR_500, _, _, _, _, _ = ifs_results('task', replay_config='aws', ifs_option='ratio', memory_budget=500)
ifsR_1K, _, _, _, _, _ = ifs_results('task', replay_config='aws', ifs_option='ratio', memory_budget=1000)
ifsR_5K, _, _, _, _, _ = ifs_results('task', replay_config='aws', ifs_option='ratio', memory_budget=5000)
ifsR_10K, _, _, _, _, _ = ifs_results('task', replay_config='aws', ifs_option='ratio', memory_budget=10000)
ifsR_15K, _, _, _, _, _ = ifs_results('task', replay_config='aws', ifs_option='ratio', memory_budget=15000)
ifsR_20K, _, _, _, _, _ = ifs_results('task', replay_config='aws', ifs_option='ratio', memory_budget=20000)

	aws-ratio-500	prec	86.4$\pm$1.7
	aws-ratio-1000	prec	84.7$\pm$1.6
	aws-ratio-5000	prec	89.2$\pm$1.6
	aws-ratio-10000	prec	89.9$\pm$1.2
	aws-ratio-15000	prec	90.4$\pm$1.3
	aws-ratio-20000	prec	90.1$\pm$1.2


In [24]:
ifsU_500, _, _, _, _, _ = ifs_results('task', replay_config='aws', ifs_option='uniform', memory_budget=500)
ifsU_1K, _, _, _, _, _ = ifs_results('task', replay_config='aws', ifs_option='uniform', memory_budget=1000)
ifsU_5K, _, _, _, _, _ = ifs_results('task', replay_config='aws', ifs_option='uniform', memory_budget=5000)
ifsU_10K, _, _, _, _, _ = ifs_results('task', replay_config='aws', ifs_option='uniform', memory_budget=10000)
ifsU_15K, _, _, _, _, _ = ifs_results('task', replay_config='aws', ifs_option='uniform', memory_budget=15000)
ifsU_20K, _, _, _, _, _ = ifs_results('task', replay_config='aws', ifs_option='uniform', memory_budget=20000)


	aws-uniform-500	prec	87.6$\pm$1.6
	aws-uniform-1000	prec	88.9$\pm$1.7
	aws-uniform-5000	prec	91.0$\pm$1.5
	aws-uniform-10000	prec	91.4$\pm$1.1
	aws-uniform-15000	prec	91.2$\pm$1.4
	aws-uniform-20000	prec	90.2$\pm$2.0


In [197]:
min_samples = 2

# ifsM_100, _, _, _, _, _ = ifs_results('class',  replay_config='ifs', ifs_option='mix', min_samples=min_samples, memory_budget=100)
# ifsM_250, _, _, _, _, _ = ifs_results('class',  replay_config='ifs', ifs_option='mix', min_samples=min_samples, memory_budget=250)
ifsM_500, _, _, _, _, _ = ifs_results('class',  replay_config='ifs', ifs_option='mix', min_samples=min_samples, memory_budget=500)
ifsM_1K, _, _, _, _, _ = ifs_results('class', replay_config='ifs', ifs_option='mix', min_samples=min_samples, memory_budget=1000)
ifsM_2K, _, _, _, _, _ = ifs_results('class', replay_config='ifs', ifs_option='mix', min_samples=min_samples, memory_budget=2000)
ifsM_3K, _, _, _, _, _ = ifs_results('class', replay_config='ifs', ifs_option='mix', min_samples=min_samples, memory_budget=3000)
ifsM_4K, _, _, _, _, _ = ifs_results('class', replay_config='ifs', ifs_option='mix', min_samples=min_samples, memory_budget=4000)
ifsM_5K, _, _, _, _, _ = ifs_results('class', replay_config='ifs', ifs_option='mix', min_samples=min_samples, memory_budget=5000)
ifsM_6K, _, _, _, _, _ = ifs_results('class', replay_config='ifs', ifs_option='mix', min_samples=min_samples, memory_budget=6000)
ifsM_7K, _, _, _, _, _ = ifs_results('class', replay_config='ifs', ifs_option='mix', min_samples=min_samples, memory_budget=7000);
ifsM_10K, _, _, _, _, _ = ifs_results('class', replay_config='ifs', ifs_option='mix', min_samples=min_samples, memory_budget=10000)
ifsM_15K, _, _, _, _, _ = ifs_results('class', replay_config='ifs', ifs_option='mix', min_samples=min_samples, memory_budget=15000)
ifsM_20K, _, _, _, _, _ = ifs_results('class', replay_config='ifs', ifs_option='mix', min_samples=min_samples, memory_budget=20000);


	ifs-mix-100	prec	67.5$\pm$0.8
	ifs-mix-250	prec	70.9$\pm$0.4
	ifs-mix-500	prec	73.3$\pm$0.4
	ifs-mix-1000	prec	76.1$\pm$0.4
	ifs-mix-2000	prec	78.9$\pm$0.3
	ifs-mix-3000	prec	80.1$\pm$0.3
	ifs-mix-4000	prec	80.8$\pm$0.2
	ifs-mix-5000	prec	81.7$\pm$0.3
	ifs-mix-6000	prec	82.0$\pm$0.3
	ifs-mix-7000	prec	82.3$\pm$0.2
	ifs-mix-10000	prec	83.0$\pm$0.2
	ifs-mix-15000	prec	83.6$\pm$0.1
	ifs-mix-20000	prec	84.1$\pm$0.1


In [10]:
min_samples = 5

# ifsM_100, _, _, _, _, _ = ifs_results('class',  replay_config='ifs', ifs_option='mix', min_samples=min_samples, memory_budget=100)
# ifsM_250, _, _, _, _, _ = ifs_results('class',  replay_config='ifs', ifs_option='mix', min_samples=min_samples, memory_budget=250)
ifsM_500, _, _, _, _, _ = ifs_results('class',  replay_config='ifs', ifs_option='mix', min_samples=min_samples, memory_budget=500)
ifsM_1K, _, _, _, _, _ = ifs_results('class', replay_config='ifs', ifs_option='mix', min_samples=min_samples, memory_budget=1000)
# ifsM_2K, _, _, _, _, _ = ifs_results('class', replay_config='ifs', ifs_option='mix', min_samples=min_samples, memory_budget=2000)
# ifsM_3K, _, _, _, _, _ = ifs_results('class', replay_config='ifs', ifs_option='mix', min_samples=min_samples, memory_budget=3000)
# ifsM_4K, _, _, _, _, _ = ifs_results('class', replay_config='ifs', ifs_option='mix', min_samples=min_samples, memory_budget=4000)
ifsM_5K, _, _, _, _, _ = ifs_results('class', replay_config='ifs', ifs_option='mix', min_samples=min_samples, memory_budget=5000)
# ifsM_6K, _, _, _, _, _ = ifs_results('class', replay_config='ifs', ifs_option='mix', min_samples=min_samples, memory_budget=6000)
# ifsM_7K, _, _, _, _, _ = ifs_results('class', replay_config='ifs', ifs_option='mix', min_samples=min_samples, memory_budget=7000);
ifsM_10K, _, _, _, _, _ = ifs_results('class', replay_config='ifs', ifs_option='mix', min_samples=min_samples, memory_budget=10000);
ifsM_15K, _, _, _, _, _ = ifs_results('class', replay_config='ifs', ifs_option='mix', min_samples=min_samples, memory_budget=15000);
ifsM_20K, _, _, _, _, _ = ifs_results('class', replay_config='ifs', ifs_option='mix', min_samples=min_samples, memory_budget=20000);


	ifs-mix-500	prec	93.1$\pm$0.6
	ifs-mix-1000	prec	93.5$\pm$0.6
	ifs-mix-5000	prec	94.5$\pm$0.8
	ifs-mix-10000	prec	94.6$\pm$1.1
	ifs-mix-15000	prec	95.7$\pm$0.4
	ifs-mix-20000	prec	94.6$\pm$0.9


In [11]:
min_samples = 10

# ifsM_100, _, _, _, _, _ = ifs_results('class',  replay_config='ifs', ifs_option='mix', min_samples=min_samples, memory_budget=100)
# ifsM_250, _, _, _, _, _ = ifs_results('class',  replay_config='ifs', ifs_option='mix', min_samples=min_samples, memory_budget=250)
ifsM_500, _, _, _, _, _ = ifs_results('class',  replay_config='ifs', ifs_option='mix', min_samples=min_samples, memory_budget=500)
ifsM_1K, _, _, _, _, _ = ifs_results('class', replay_config='ifs', ifs_option='mix', min_samples=min_samples, memory_budget=1000)
# ifsM_2K, _, _, _, _, _ = ifs_results('class', replay_config='ifs', ifs_option='mix', min_samples=min_samples, memory_budget=2000)
# ifsM_3K, _, _, _, _, _ = ifs_results('class', replay_config='ifs', ifs_option='mix', min_samples=min_samples, memory_budget=3000)
# ifsM_4K, _, _, _, _, _ = ifs_results('class', replay_config='ifs', ifs_option='mix', min_samples=min_samples, memory_budget=4000)
ifsM_5K, _, _, _, _, _ = ifs_results('class', replay_config='ifs', ifs_option='mix', min_samples=min_samples, memory_budget=5000)
# ifsM_6K, _, _, _, _, _ = ifs_results('class', replay_config='ifs', ifs_option='mix', min_samples=min_samples, memory_budget=6000)
# ifsM_7K, _, _, _, _, _ = ifs_results('class', replay_config='ifs', ifs_option='mix', min_samples=min_samples, memory_budget=7000);
ifsM_10K, _, _, _, _, _ = ifs_results('class', replay_config='ifs', ifs_option='mix', min_samples=min_samples, memory_budget=10000);
ifsM_15K, _, _, _, _, _ = ifs_results('class', replay_config='ifs', ifs_option='mix', min_samples=min_samples, memory_budget=15000);
ifsM_20K, _, _, _, _, _ = ifs_results('class', replay_config='ifs', ifs_option='mix', min_samples=min_samples, memory_budget=20000);


	ifs-mix-500	prec	94.0$\pm$1.4
	ifs-mix-1000	prec	94.9$\pm$0.7
	ifs-mix-5000	prec	94.2$\pm$0.9
	ifs-mix-10000	prec	95.1$\pm$0.8
	ifs-mix-15000	prec	94.5$\pm$1.0
	ifs-mix-20000	prec	95.1$\pm$0.6


In [12]:
min_samples = 15

# ifsM_100, _, _, _, _, _ = ifs_results('class',  replay_config='ifs', ifs_option='mix', min_samples=min_samples, memory_budget=100)
# ifsM_250, _, _, _, _, _ = ifs_results('class',  replay_config='ifs', ifs_option='mix', min_samples=min_samples, memory_budget=250)
ifsM_500, _, _, _, _, _ = ifs_results('class',  replay_config='ifs', ifs_option='mix', min_samples=min_samples, memory_budget=500)
ifsM_1K, _, _, _, _, _ = ifs_results('class', replay_config='ifs', ifs_option='mix', min_samples=min_samples, memory_budget=1000)
# ifsM_2K, _, _, _, _, _ = ifs_results('class', replay_config='ifs', ifs_option='mix', min_samples=min_samples, memory_budget=2000)
# ifsM_3K, _, _, _, _, _ = ifs_results('class', replay_config='ifs', ifs_option='mix', min_samples=min_samples, memory_budget=3000)
# ifsM_4K, _, _, _, _, _ = ifs_results('class', replay_config='ifs', ifs_option='mix', min_samples=min_samples, memory_budget=4000)
ifsM_5K, _, _, _, _, _ = ifs_results('class', replay_config='ifs', ifs_option='mix', min_samples=min_samples, memory_budget=5000)
# ifsM_6K, _, _, _, _, _ = ifs_results('class', replay_config='ifs', ifs_option='mix', min_samples=min_samples, memory_budget=6000)
# ifsM_7K, _, _, _, _, _ = ifs_results('class', replay_config='ifs', ifs_option='mix', min_samples=min_samples, memory_budget=7000);
ifsM_10K, _, _, _, _, _ = ifs_results('class', replay_config='ifs', ifs_option='mix', min_samples=min_samples, memory_budget=10000);
ifsM_15K, _, _, _, _, _ = ifs_results('class', replay_config='ifs', ifs_option='mix', min_samples=min_samples, memory_budget=15000);
ifsM_20K, _, _, _, _, _ = ifs_results('class', replay_config='ifs', ifs_option='mix', min_samples=min_samples, memory_budget=20000);


	ifs-mix-500	prec	93.5$\pm$1.5
	ifs-mix-1000	prec	94.7$\pm$0.9
	ifs-mix-5000	prec	94.2$\pm$1.0
	ifs-mix-10000	prec	94.7$\pm$0.2
	ifs-mix-15000	prec	94.4$\pm$1.3
	ifs-mix-20000	prec	95.9$\pm$0.9


In [13]:
min_samples = 20

# ifsM_100, _, _, _, _, _ = ifs_results('class',  replay_config='ifs', ifs_option='mix', min_samples=min_samples, memory_budget=100)
# ifsM_250, _, _, _, _, _ = ifs_results('class',  replay_config='ifs', ifs_option='mix', min_samples=min_samples, memory_budget=250)
ifsM_500, _, _, _, _, _ = ifs_results('class',  replay_config='ifs', ifs_option='mix', min_samples=min_samples, memory_budget=500)
ifsM_1K, _, _, _, _, _ = ifs_results('class', replay_config='ifs', ifs_option='mix', min_samples=min_samples, memory_budget=1000)
# ifsM_2K, _, _, _, _, _ = ifs_results('class', replay_config='ifs', ifs_option='mix', min_samples=min_samples, memory_budget=2000)
# ifsM_3K, _, _, _, _, _ = ifs_results('class', replay_config='ifs', ifs_option='mix', min_samples=min_samples, memory_budget=3000)
# ifsM_4K, _, _, _, _, _ = ifs_results('class', replay_config='ifs', ifs_option='mix', min_samples=min_samples, memory_budget=4000)
ifsM_5K, _, _, _, _, _ = ifs_results('class', replay_config='ifs', ifs_option='mix', min_samples=min_samples, memory_budget=5000)
# ifsM_6K, _, _, _, _, _ = ifs_results('class', replay_config='ifs', ifs_option='mix', min_samples=min_samples, memory_budget=6000)
# ifsM_7K, _, _, _, _, _ = ifs_results('class', replay_config='ifs', ifs_option='mix', min_samples=min_samples, memory_budget=7000);
ifsM_10K, _, _, _, _, _ = ifs_results('class', replay_config='ifs', ifs_option='mix', min_samples=min_samples, memory_budget=10000);
ifsM_15K, _, _, _, _, _ = ifs_results('class', replay_config='ifs', ifs_option='mix', min_samples=min_samples, memory_budget=15000);
ifsM_20K, _, _, _, _, _ = ifs_results('class', replay_config='ifs', ifs_option='mix', min_samples=min_samples, memory_budget=20000);



	ifs-mix-500	prec	93.6$\pm$1.0
	ifs-mix-1000	prec	93.7$\pm$1.3
	ifs-mix-5000	prec	94.5$\pm$0.9
	ifs-mix-10000	prec	94.7$\pm$0.7
	ifs-mix-15000	prec	95.1$\pm$0.7
	ifs-mix-20000	prec	94.7$\pm$0.9


In [207]:
ifsR_100, _, _, _, _, _ = ifs_results('class', replay_config='aws', ifs_option='ratio', memory_budget=100)
ifsR_250, _, _, _, _, _ = ifs_results('class', replay_config='aws', ifs_option='ratio', memory_budget=250)
ifsR_500, _, _, _, _, _ = ifs_results('class', replay_config='aws', ifs_option='ratio', memory_budget=500)
ifsR_1K, _, _, _, _, _ = ifs_results('class', replay_config='aws', ifs_option='ratio', memory_budget=1000)
ifsR_2K, _, _, _, _, _ = ifs_results('class', replay_config='aws', ifs_option='ratio', memory_budget=2000)
ifsR_3K, _, _, _, _, _ = ifs_results('class', replay_config='aws', ifs_option='ratio', memory_budget=3000)
ifsR_4K, _, _, _, _, _ = ifs_results('class', replay_config='aws', ifs_option='ratio', memory_budget=4000)
ifsR_5K, _, _, _, _, _ = ifs_results('class', replay_config='aws', ifs_option='ratio', memory_budget=5000)
ifsR_6K, _, _, _, _, _ = ifs_results('class', replay_config='aws', ifs_option='ratio', memory_budget=6000)
ifsR_7K, _, _, _, _, _ = ifs_results('class', replay_config='aws', ifs_option='ratio', memory_budget=7000)
ifsR_10K, _, _, _, _, _ = ifs_results('class', replay_config='aws', ifs_option='ratio', memory_budget=10000)
ifsR_15K, _, _, _, _, _ = ifs_results('class', replay_config='aws', ifs_option='ratio', memory_budget=15000)
ifsR_20K, _, _, _, _, _ = ifs_results('class', replay_config='aws', ifs_option='ratio', memory_budget=20000)

	aws-ratio-100	prec	67.9$\pm$0.3
	aws-ratio-250	prec	70.5$\pm$0.2
	aws-ratio-500	prec	72.7$\pm$0.5
	aws-ratio-1000	prec	76.0$\pm$0.2
	aws-ratio-2000	prec	78.8$\pm$0.2
	aws-ratio-3000	prec	80.2$\pm$0.5
	aws-ratio-4000	prec	80.9$\pm$0.1
	aws-ratio-5000	prec	81.7$\pm$0.2
	aws-ratio-6000	prec	81.7$\pm$0.2
	aws-ratio-7000	prec	82.3$\pm$0.1
	aws-ratio-10000	prec	83.2$\pm$0.1
	aws-ratio-15000	prec	83.9$\pm$0.1
	aws-ratio-20000	prec	84.5$\pm$0.2


In [208]:
ifsU_100, _, _, _, _, _ = ifs_results('class', replay_config='aws', ifs_option='uniform', memory_budget=100)
ifsU_250, _, _, _, _, _ = ifs_results('class', replay_config='aws', ifs_option='uniform', memory_budget=250)
ifsU_500, _, _, _, _, _ = ifs_results('class', replay_config='aws', ifs_option='uniform', memory_budget=500)
ifsU_1K, _, _, _, _, _ = ifs_results('class', replay_config='aws', ifs_option='uniform', memory_budget=1000)
ifsU_2K, _, _, _, _, _ = ifs_results('class', replay_config='aws', ifs_option='uniform', memory_budget=2000)
ifsU_3K, _, _, _, _, _ = ifs_results('class', replay_config='aws', ifs_option='uniform', memory_budget=3000)
ifsU_4K, _, _, _, _, _ = ifs_results('class', replay_config='aws', ifs_option='uniform', memory_budget=4000)
ifsU_5K, _, _, _, _, _ = ifs_results('class', replay_config='aws', ifs_option='uniform', memory_budget=5000)
ifsU_6K, _, _, _, _, _ = ifs_results('class', replay_config='aws', ifs_option='uniform', memory_budget=6000)
ifsU_7K, _, _, _, _, _ = ifs_results('class', replay_config='aws', ifs_option='uniform', memory_budget=7000)
ifsU_10K, _, _, _, _, _ = ifs_results('class', replay_config='aws', ifs_option='uniform', memory_budget=10000)
ifsU_15K, _, _, _, _, _ = ifs_results('class', replay_config='aws', ifs_option='uniform', memory_budget=15000)
ifsU_20K, _, _, _, _, _ = ifs_results('class', replay_config='aws', ifs_option='uniform', memory_budget=20000)


	aws-uniform-100	prec	65.5$\pm$0.7
	aws-uniform-250	prec	72.8$\pm$0.2
	aws-uniform-500	prec	76.4$\pm$0.4
	aws-uniform-1000	prec	79.7$\pm$0.4
	aws-uniform-2000	prec	82.1$\pm$0.3
	aws-uniform-3000	prec	83.0$\pm$0.3
	aws-uniform-4000	prec	83.7$\pm$0.2
	aws-uniform-5000	prec	84.1$\pm$0.1
	aws-uniform-6000	prec	86.2$\pm$2.3
	aws-uniform-7000	prec	84.8$\pm$0.0
	aws-uniform-10000	prec	85.3$\pm$0.2
	aws-uniform-15000	prec	85.8$\pm$0.3
	aws-uniform-20000	prec	86.2$\pm$0.2


/home/mr6564/anaconda3/envs/PyTorch/lib/python3.7/site-packages/ipykernel_launcher.py:143: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
